In [1]:
#Import Libraries
import jaydebeapi
import pandas as pd
import matplotlib
import psycopg2
import csv
from getpass import getpass
import re

In [ ]:
def serverDetails():
    try:
        print("*****Enter the server details*****\n")
        host = input("Enter the host name: ")
        port = input("Enter the port name: ")
        uid = input("Enter the username name: ")
        pwd = str(getpass("Enter the password: "))
        # Data Virtuality  class name
        driver_class = "com.zetaris.lightning.jdbc.LightningDriver"

        # Data Virtuality Server driver file (full path)
        driver_file = "ndp-jdbc-driver-2.1.0.10-driver.jar"

        # JDBC connection string
        connection_string='jdbc:zetaris:lightning@'+ host +':'+ port

        # Establish JDBC connection
        con = jaydebeapi.connect(driver_class, connection_string, [uid, pwd], driver_file,)
        print("\n Connection successfull")  
        return con
    except:
        print("Entered details are incorrect.Cannot connect to database server.\n")
        print("Pleas try again.\n")
        serverDetails()
            
    
def mainMenu(con):
    print("\n Select the following option to migrate: \n")
    print("1. Data Pipeline container")
    print("2. Individual Data Pipeline")
    print("3. Data Quality container")
    print("4. Individual Data Quality Pipeline")
    print("5. Data Mart")
    print("6. Individual Table/View in the existing Datamart")
    print("7. Permanent Views")
    print("8. Exit")
    try:
        option = int(input("\nEnter your selection: "))
        if option == 1:
            #DataPipelineContainer_InsertQueries(con)
            DataPipelineContainer_InsertQueries(con,option)
        elif option == 2:
            #IndivdualDataPipeline_InsertQueries(con)
            IndivdualDataPipeline_InsertQueries(con,option)
        elif option == 3:
            #DQContainer_InsertQueries(con)
            DataPipelineContainer_InsertQueries(con,option)
        elif option == 4:
            #DQIndividualPipeline_InsertQueries(con)
            IndivdualDataPipeline_InsertQueries(con,option)
        elif option == 5:
            DataMart_InsertQueries(con)
        elif option == 6:
            DataMart_IndTable_InsertQueries(con)
        elif option == 7:
            PermanentViews_InsertQueries(con)
        elif option == 8:
            print("\nThanks for using the system")
            exit
        else:
            print("\n invalid option\n")
            ans=input("Do you want to continue?(y/n): ")
            ans.lower()
            if ans=='y':
                print("\n")
                mainMenu(con)
            else:
                print("\n Thanks for using the system")
                exit
    except:
        print("\nIndvalid selection. Please select between 1-6.\n")
        mainMenu(con)

def checkcontainername(container_name,con):
    sql_containername_check="""select 'INSERT INTO pipeline_container (id, case_sensitive_name,description, name, fk_organisation_id)  VALUES (' || id || ',''' || CHAR(39)||COALESCE(case_sensitive_name, '')||CHAR(39)|| ''',''' ||CHAR(39)||COALESCE(description, '')||CHAR(39) || ''',''' ||CHAR(39)||COALESCE(name,'')||CHAR(39) || ''',' || COALESCE(fk_organisation_id, 0) ||''');'  from metastore.pipeline_container where  name='%s'"""%(container_name)
    curs = con.cursor()
    curs.execute(sql_containername_check)
    res = curs.fetchall()
    if len(res)>0:
        return True
    else:
        return False
    
def checkcontainer_pipeline_name(container_name,pipeline_name, con):
    #sql_containername_check="""select 'INSERT INTO pipeline_container  (id, case_sensitive_name,description, name, fk_organisation_id)  VALUES (' || id || ',''' || CHAR(39)||COALESCE(case_sensitive_name, '')||CHAR(39)|| ''',''' ||CHAR(39)||COALESCE(description, '')||CHAR(39) || ''',''' ||CHAR(39)||COALESCE(name,'')||CHAR(39) || ''',' || COALESCE(fk_organisation_id, 0) ||''');'  from metastore.pipeline_container where  name='%s'"""%(container_name)
    #curs = con.cursor()
    #curs.execute(sql_containername_check)
    #res_con = curs.fetchall()
    sql_container_pipeline_chk="""select 'INSERT INTO pipeline_relation (id, case_sensitive_name,description, name, fk_pipeline_container_id)VALUES(' || id || ',''' ||CHAR(39)|| COALESCE(case_sensitive_name, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(description, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(name, '')||CHAR(39) || ''',' || COALESCE(fk_pipeline_container_id, 0) || ');'     from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s') and case_sensitive_name = '%s'"""%(container_name, pipeline_name)
    curs = con.cursor()
    curs.execute(sql_container_pipeline_chk)
    res_con_pipe = curs.fetchall()
    if len(res_con_pipe)>0:
        return True
    else:
        return False
    
def chk_datamart_name(datamart_name,con):
    sql_data_mart_ck = """SELECT 'INSERT INTO data_mart (id, case_sensitive_name ,default_view , name , fk_organisation_id ) VALUES (' || id || ',''' || CHAR(39)|| COALESCE(case_sensitive_name, '') ||CHAR(39)|| ''',''' ||CHAR(39)|| COALESCE(default_view, '')||CHAR(39)||''',''' ||CHAR(39)|| COALESCE(name,'')||CHAR(39) ||''','|| COALESCE(fk_organisation_id,'') ||');' FROM metastore.data_mart where name= '%s';"""%(datamart_name)
    curs = con.cursor()
    curs.execute(sql_data_mart_ck)
    res_dm_chk = curs.fetchall()
    if len(res_dm_chk)>0:
        return True
    else:
        return False   
    
def chk_datamart_table_name(datamart_name,table_name, con):
    sql_data_mart_ck = """SELECT 'INSERT INTO data_mart_table (id, case_sensitive_name , name , source_table  ,fk_data_mart_id  )  VALUES (' || id || ',''' || CHAR(39) || COALESCE(case_sensitive_name, '') || CHAR(39) || ''',''' || CHAR(39) || COALESCE(name, '') || CHAR(39) || ''',''' || CHAR(39) || COALESCE(source_table, '') || CHAR(39) || ''',' || COALESCE(fk_data_mart_id, 0) || ');' FROM metastore.data_mart_table where fk_data_mart_id in ( select id from metastore.data_mart where name = '%s') and name='%s';"""%(datamart_name,table_name) 
    curs = con.cursor()
    curs.execute(sql_data_mart_ck)
    res_dm_chk = curs.fetchall()
    if len(res_dm_chk)>0:
        return True
    else:
        return False  
    
def chk_view_name(View_name,con):
    
    sql_view_name_ck = """select 'INSERT INTO schema_store_view (id, description ,generator, materialized_table,name,query,fk_organisation_id   )  VALUES (' || id || ',''' || CHAR(39) || COALESCE(description, '') || CHAR(39) || ''',''' || CHAR(39) || COALESCE(generator, '') || char(39) || ''',''' || CHAR(39) || COALESCE(materialized_table, '') || CHAR(39) || ''',''' || CHAR(39) || COALESCE(name, '') || CHAR(39) || ''',''' || CHAR(39) || COALESCE(query, '') || CHAR(39) || ''',' || COALESCE(fk_organisation_id, 0) || ');' FROM metastore.schema_store_view where name = '%s'"""%(View_name)
    curs = con.cursor()
    curs.execute(sql_view_name_ck)
    res_view_chk = curs.fetchall()
    if len(res_view_chk)>0:
        return True
    else:
        return False   
    
def DataPipelineContainer_InsertQueries(con,option):
    container_name = (input("\nEnter the container name: "))
    container_name = container_name.lower()
    chk_container_name= checkcontainername(container_name,con)
    if chk_container_name == False:
        print("\nIncorrect container name. Please check the container name & try again.")  
        mainMenu(con)
    else:
        curs = con.cursor()
        list_queries = []
        print("\n--pipeline_container--")
        #sql_pipeline_container = """ select 'INSERT INTO pipeline_container  (id, case_sensitive_name,description, name, fk_organisation_id)  VALUES (' || id || ',''' || COALESCE(case_sensitive_name, '') || ''',''' || COALESCE(description, '') || ''',''' || COALESCE(name,'') || ''',' || COALESCE(fk_organisation_id, 0) ||''');'  from metastore.pipeline_container where  name='%s'"""%(container_name)
        sql_pipeline_container = """select 'INSERT INTO pipeline_container  (id, case_sensitive_name,description, name, fk_organisation_id)  VALUES (' || id || ',''' || CHAR(39)||COALESCE(case_sensitive_name, '')||CHAR(39)|| ''',''' ||CHAR(39)||COALESCE(description, '')||CHAR(39) || ''',''' ||CHAR(39)||COALESCE(name,'')||CHAR(39) || ''',' || COALESCE(fk_organisation_id, 0) ||''');'  from metastore.pipeline_container where  name='%s'"""%(container_name)
        curs.execute(sql_pipeline_container)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])
        print("\n--pipeline_node_relation--")
        #sql_pipeline_relation = """select 'INSERT INTO pipeline_relation (id, case_sensitive_name,description, name, fk_pipeline_container_id)
         #VALUES (' || id || ',''' || COALESCE(case_sensitive_name, '') || ''',''' || COALESCE(description, '') || ''',''' || COALESCE(name, '') || ''',' || COALESCE(fk_pipeline_container_id, 0) || ');'
        #from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s')"""%(container_name)
        sql_pipeline_relation = """select 'INSERT INTO pipeline_relation (id, case_sensitive_name,description, name, fk_pipeline_container_id) VALUES (' || id || ',''' || CHAR(39)||COALESCE(case_sensitive_name, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(description, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(name, '')||CHAR(39) || ''',' || COALESCE(fk_pipeline_container_id, 0) || ');' from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s')"""%(container_name)
        curs.execute(sql_pipeline_relation)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])
        print("\n--pipeline_node--")
        #sql_pipeline_node="""select 'INSERT INTO pipeline_node (id, case_sensitive_name,description, name, fk_pipeline_relation_id, pipeline_type)  VALUES (' || id || ',''' || COALESCE(case_sensitive_name, '') || ''',''' || COALESCE(description, '') || ''',''' || COALESCE(name,'') || ''',' || COALESCE(fk_pipeline_relation_id, 0) ||',''' || COALESCE(pipeline_type,'') ||''');'  from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s'))"""%(container_name)
        sql_pipeline_node="""select 'INSERT INTO pipeline_node (id, case_sensitive_name,description, name, fk_pipeline_relation_id, pipeline_type)  VALUES (' || id || ',''' ||CHAR(39)|| COALESCE(case_sensitive_name, '')||CHAR(39) || ''',''' ||CHAR(39)||COALESCE(description, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(name,'')||CHAR(39) || ''',' || COALESCE(fk_pipeline_relation_id, 0) ||',''' ||char(39)|| COALESCE(pipeline_type,'')||char(39) ||''');'  from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s'))"""%(container_name)
        curs.execute(sql_pipeline_node)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])
        print("\n--pipeline_node_schema--")
        #sql_pipeline_node_schema="""select 'INSERT INTO pipeline_node_schema (id, column_alias,column_name, data_type, sql_expression, fk_pipeline_node_id)  VALUES (' || id || ',''' || COALESCE(column_alias, '') || ''',''' || COALESCE(column_name, '') || ''',''' || COALESCE(data_type,'') || ''',''' || COALESCE(sql_expression, '') ||''',' || COALESCE(fk_pipeline_node_id, 0) ||');'  FROM metastore.pipeline_node_schema where fk_pipeline_node_id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s')))"""%(container_name)
        sql_pipeline_node_schema="""select 'INSERT INTO pipeline_node_schema (id, column_alias,column_name, data_type, sql_expression, fk_pipeline_node_id)  VALUES (' || id || ',''' ||CHAR(39)|| COALESCE(column_alias, '')||CHAR(39)|| ''',''' ||CHAR(39)|| COALESCE(column_name, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(data_type,'')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(sql_expression, '')||CHAR(39) ||''',' || COALESCE(fk_pipeline_node_id, 0) ||');'  FROM metastore.pipeline_node_schema where fk_pipeline_node_id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s')))"""%(container_name)
        curs.execute(sql_pipeline_node_schema)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])
        print("\n--pipeline_node_property--")
        #sql_pipeline_node_property="""select 'INSERT INTO pipeline_node_property (id, property_key,property_value, fk_pipeline_node_id)  VALUES (' || id || ',''' || COALESCE(property_key, '') || ''',''' || COALESCE(property_value, '') ||''',' || COALESCE(fk_pipeline_node_id, 0) ||');'  FROM metastore.pipeline_node_property  where fk_pipeline_node_id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s')))"""%(container_name)
        sql_pipeline_node_property="""select 'INSERT INTO pipeline_node_property (id, property_key,property_value, fk_pipeline_node_id)  VALUES (' || id || ',''' ||CHAR(39)||COALESCE(property_key, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(property_value, '')||CHAR(39) ||''',' || COALESCE(fk_pipeline_node_id, 0) ||');'  FROM metastore.pipeline_node_property  where fk_pipeline_node_id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s')))"""%(container_name)
        curs.execute(sql_pipeline_node_property)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])


        print("\n--Datasource_Node--")
        #sql_datasource_Node="""select 'INSERT INTO pipeline_datasource(id, datasource,datasource_table) VALUES(' || id || ',''' || COALESCE(datasource, '') || ''',''' || COALESCE(datasource_table, '') || ''');' FROM metastore.pipeline_datasource where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s')))"""%(container_name)
        sql_datasource_Node="""select 'INSERT INTO pipeline_datasource(id, datasource,datasource_table) VALUES(' || id || ',''' ||CHAR(39)|| COALESCE(datasource, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(datasource_table, '')||CHAR(39) || ''');' FROM metastore.pipeline_datasource where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s')))"""%(container_name) 
        curs.execute(sql_datasource_Node)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--sql_table--")
        #sql_sql_table="""select 'INSERT INTO pipeline_sqltable (id, sql_query,source_tables) VALUES(' || id || ',''' || COALESCE(sql_query, '') || ''',''' || COALESCE(source_tables, '') || ''');' FROM metastore.pipeline_sqltable where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s')))"""%(container_name)
        sql_sql_table="""select 'INSERT INTO pipeline_sqltable (id, sql_query,source_tables)
        VALUES(' || id || ',''' ||CHAR(39)||COALESCE(sql_query, '')||CHAR(39) || ''',''' ||CHAR(39)|| 
        COALESCE(source_tables, '')||CHAR(39) || ''');' FROM metastore.pipeline_sqltable where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s')))"""%(container_name)
        curs.execute(sql_sql_table)
        res = curs.fetchall()
        to_substitute = "''"
        for i in range(len(res)):
            list_queries.append(res[i][0])


        print("\n--projection_node--")
        #sql_projection_node="""select 'INSERT INTO pipeline_projection (id, filter_expression,order_by, windows_spec)  VALUES (' || id || ',''' || COALESCE(filter_expression, '') || ''',''' || COALESCE(order_by, '') ||''',''' || COALESCE(windows_spec,'') ||''');'  FROM metastore.pipeline_projection where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s')))"""%(container_name)
        sql_projection_node="""select 'INSERT INTO pipeline_projection (id, filter_expression,order_by, windows_spec)  VALUES (' || id || ',''' ||CHAR(39)|| COALESCE(filter_expression, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(order_by, '')||CHAR(39) ||''',''' ||CHAR(39)|| COALESCE(windows_spec,'')||CHAR(39)||''');'  FROM metastore.pipeline_projection where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s')))"""%(container_name)
        curs.execute(sql_projection_node)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--join_node--")
        #sql_join_node="""select 'INSERT INTO pipeline_join (id, filter_expression,join_predicate, join_tables, order_by)  VALUES (' || id || ',''' || COALESCE(filter_expression, '') || ''',''' || COALESCE(join_predicate, '') ||''',''' || COALESCE(join_tables,'') ||''',''' || COALESCE(order_by,'') ||''');'  FROM metastore.pipeline_join where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name='%s')))"""%(container_name)
        sql_join_node="""select 'INSERT INTO pipeline_join (id, filter_expression,join_predicate, join_tables, order_by)  VALUES (' || id || ',''' ||CHAR(39)|| COALESCE(filter_expression, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(join_predicate, '')||CHAR(39) ||''',''' ||CHAR(39)|| COALESCE(join_tables,'')||CHAR(39) ||''',''' || CHAR(39)||COALESCE(order_by,'')||CHAR(39) ||''');'  FROM metastore.pipeline_join where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name='%s')))"""%(container_name)
        curs.execute(sql_join_node)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--aggregation_node--")
        #sql_aggregation_node="""select 'INSERT INTO pipeline_aggregation (id, filter_expression,group_expression, having_expression, order_by)  VALUES (' || id || ',''' || COALESCE(filter_expression, '') || ''',''' || COALESCE(group_expression, '') ||''',''' || COALESCE(having_expression,'') ||''',''' || COALESCE(order_by,'') ||''');'  FROM metastore.pipeline_aggregation where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s')))"""%(container_name)
        sql_aggregation_node="""select 'INSERT INTO pipeline_aggregation (id, filter_expression,group_expression, having_expression, order_by)  VALUES (' || id || ',''' ||CHAR(39)||COALESCE(filter_expression, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(group_expression, '')||CHAR(39) ||''',''' ||CHAR(39)|| COALESCE(having_expression,'')||CHAR(39) ||''',''' ||CHAR(39)|| COALESCE(order_by,'')||CHAR(39) ||''');'  FROM metastore.pipeline_aggregation where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s')))"""%(container_name)
        curs.execute(sql_aggregation_node)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--sink_node--")
        sql_aggregation_node="""select 'INSERT INTO pipeline_sink (id, sink_type)  VALUES (' || id || ',''' ||char(39)|| COALESCE(sink_type, '')||char(39) ||''');'  FROM metastore.pipeline_sink where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s')))"""%(container_name)
        curs.execute(sql_aggregation_node)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])
        
        if(option == 3):
            print("\n--DQ_pipeline--")
            sql_DQ_pipeline = """select 'INSERT INTO pipeline_simple_dq (id) VALUES ('||id||');' FROM metastore.pipeline_simple_dq where id in (select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name='%s')))"""%(container_name)
            curs.execute(sql_DQ_pipeline)
            res = curs.fetchall()
            for i in range(len(res)):
                list_queries.append(res[i][0])
            print("\n--pipeline_simple_dq_rule--")
            #sql_pipeline_simple_dq_rule = """SELECT 'INSERT INTO pipeline_simple_dq_rule (id, columns ,expression , filter , name,fk_pipeline_simple_dq_id  ) VALUES (' || id || ',''' || COALESCE(columns, '') || ''',''' || COALESCE(expression, '') || ''',''' || COALESCE(filter, '') || ''',''' || COALESCE(name, '') || ''',' || COALESCE(fk_pipeline_simple_dq_id, 0) || ');' FROM metastore.pipeline_simple_dq_rule where fk_pipeline_simple_dq_id in (select id from metastore.pipeline_simple_dq where id in (select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s'))))"""%(container_name)
            sql_pipeline_simple_dq_rule = """SELECT 'INSERT INTO pipeline_simple_dq_rule (id, columns ,expression , filter , name,fk_pipeline_simple_dq_id  ) VALUES (' || id || ',''' ||CHAR(39)||COALESCE(columns, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(expression, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(filter, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(name, '')||CHAR(39) || ''',' || COALESCE(fk_pipeline_simple_dq_id, 0) || ');' FROM metastore.pipeline_simple_dq_rule where fk_pipeline_simple_dq_id in (select id from metastore.pipeline_simple_dq where id in (select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s'))))"""%(container_name)
            curs.execute(sql_pipeline_simple_dq_rule)
            res = curs.fetchall()
            for i in range(len(res)):
                list_queries.append(res[i][0])

        for i in range(len(list_queries)):
            print("\n")
            print(list_queries[i])
                                    
        generateSQLfile(list_queries,container_name,'','Pipeline')
        mainMenu(con)

def IndivdualDataPipeline_InsertQueries(con,option):
   
    container_name = (input("Enter the container name: "))
    pipeline_name = (input("Enter the pipeline name: "))
    chk_con_pipe_name= checkcontainer_pipeline_name(container_name,pipeline_name, con)
    if chk_con_pipe_name == False:
        print("\n Incorrect container or pipeline name. Please check the container/pipeline name & try again")  
        mainMenu(con)
    else:
        curs = con.cursor()
        list_queries = []
        print("\n--pipeline_node_relation--")
        #sql_pipeline_relation = """select 'INSERT INTO pipeline_relation (id, case_sensitive_name,description, name, fk_pipeline_container_id)
        # VALUES (' || id || ',''' || COALESCE(case_sensitive_name, '') || ''',''' || COALESCE(description, '') || ''',''' || COALESCE(name, '') || ''',' || COALESCE(fk_pipeline_container_id, 0) || ');'
        #from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s') and case_sensitive_name = '%s'"""%(container_name, pipeline_name)
        sql_pipeline_relation = """select 'INSERT INTO pipeline_relation (id, case_sensitive_name,description, name, fk_pipeline_container_id)VALUES(' || id || ',''' ||CHAR(39)|| COALESCE(case_sensitive_name, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(description, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(name, '')||CHAR(39) || ''',' || COALESCE(fk_pipeline_container_id, 0) || ');'     from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s') and case_sensitive_name = '%s'"""%(container_name, pipeline_name)
        curs.execute(sql_pipeline_relation)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--pipeline_node--")
        #sql_pipeline_node="""select 'INSERT INTO pipeline_node (id, case_sensitive_name,description, name, fk_pipeline_relation_id, pipeline_type)  VALUES (' || id || ',''' || COALESCE(case_sensitive_name, '') || ''',''' || COALESCE(description, '') || ''',''' || COALESCE(name,'') || ''',' || COALESCE(fk_pipeline_relation_id, 0) ||',''' || COALESCE(pipeline_type,'') ||''');'  from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        #sql_pipeline_node="""select 'INSERT INTO pipeline_node (id, case_sensitive_name,description, name, fk_pipeline_relation_id, pipeline_type)  VALUES (' || id || ',''' ||CHAR(39)|| COALESCE(case_sensitive_name, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(description, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(name,'')||CHAR(39) || ''',' || COALESCE(fk_pipeline_relation_id, 0) ||',''' ||CHAR(39)|| COALESCE(pipeline_type,'')||CHAR(39) ||''');'  from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s') and case_sensitive_name = '%s')"""%(container_name, pipeline_name)
        sql_pipeline_node="""select 'INSERT INTO pipeline_node (id, case_sensitive_name,description, name, fk_pipeline_relation_id, pipeline_type)  VALUES (' || id || ',''' ||CHAR(39)|| COALESCE(case_sensitive_name, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(description, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(name,'')||CHAR(39) || ''',' || COALESCE(fk_pipeline_relation_id, 0) ||',''' ||CHAR(39)|| COALESCE(pipeline_type,'')||CHAR(39) ||''');'  from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s') and case_sensitive_name = '%s')"""%(container_name, pipeline_name)
        curs.execute(sql_pipeline_node)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--pipeline_node_schema--")
        #sql_pipeline_node_schema="""select 'INSERT INTO pipeline_node_schema (id, column_alias,column_name, data_type, sql_expression, fk_pipeline_node_id)  VALUES (' || id || ',''' || COALESCE(column_alias, '') || ''',''' || COALESCE(column_name, '') || ''',''' || COALESCE(data_type,'') || ''',''' || COALESCE(sql_expression, '') ||''',' || COALESCE(fk_pipeline_node_id, 0) ||');'  FROM metastore.pipeline_node_schema where fk_pipeline_node_id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        sql_pipeline_node_schema="""select 'INSERT INTO pipeline_node_schema (id, column_alias,column_name, data_type, sql_expression, fk_pipeline_node_id)  VALUES (' || id || ',''' ||CHAR(39)|| COALESCE(column_alias, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(column_name, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(data_type,'')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(sql_expression, '')||CHAR(39) ||''',' || COALESCE(fk_pipeline_node_id, 0) ||');'  FROM metastore.pipeline_node_schema where fk_pipeline_node_id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        curs.execute(sql_pipeline_node_schema)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])
        print("\n--pipeline_node_property--")
        #sql_pipeline_node_property="""select 'INSERT INTO pipeline_node_property (id, property_key,property_value, fk_pipeline_node_id)  VALUES (' || id || ',''' || COALESCE(property_key, '') || ''',''' || COALESCE(property_value, '') ||''',' || COALESCE(fk_pipeline_node_id, 0) ||');'  FROM metastore.pipeline_node_property  where fk_pipeline_node_id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        sql_pipeline_node_property="""select 'INSERT INTO pipeline_node_property (id, property_key,property_value, fk_pipeline_node_id)  VALUES (' || id || ',''' ||CHAR(39)|| COALESCE(property_key, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(property_value, '')||CHAR(39) ||''',' || COALESCE(fk_pipeline_node_id, 0) ||');'  FROM metastore.pipeline_node_property  where fk_pipeline_node_id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        curs.execute(sql_pipeline_node_property)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--Datasource_Node--")
        #sql_datasource_Node="""select 'INSERT INTO pipeline_datasource(id, datasource,datasource_table) VALUES(' || id || ',''' || COALESCE(datasource, '') || ''',''' || COALESCE(datasource_table, '') || ''');' FROM metastore.pipeline_datasource where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        sql_datasource_Node="""select 'INSERT INTO pipeline_datasource(id, datasource,datasource_table) VALUES(' || id || ',''' ||CHAR(39)|| COALESCE(datasource, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(datasource_table, '')||CHAR(39) || ''');' FROM metastore.pipeline_datasource where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        curs.execute(sql_datasource_Node)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--sql_table--")
        #sql_sql_table="""select 'INSERT INTO pipeline_sqltable (id, sql_query,source_tables) VALUES(' || id || ',''' || COALESCE(sql_query, '') || ''',''' || COALESCE(source_tables, '') || ''');' FROM metastore.pipeline_sqltable where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        sql_sql_table="""select 'INSERT INTO pipeline_sqltable (id, sql_query,source_tables) VALUES(' || id || ',''' ||CHAR(39)||COALESCE(sql_query, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(source_tables, '')||CHAR(39) || ''');' FROM metastore.pipeline_sqltable where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        curs.execute(sql_sql_table)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--projection_node--")
        #sql_projection_node="""select 'INSERT INTO pipeline_projection (id, filter_expression,order_by, windows_spec)  VALUES (' || id || ',''' || COALESCE(filter_expression, '') || ''',''' || COALESCE(order_by, '') ||''',''' || COALESCE(windows_spec,'') ||''');'  FROM metastore.pipeline_projection where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        sql_projection_node="""select 'INSERT INTO pipeline_projection (id, filter_expression,order_by, windows_spec)  VALUES (' || id || ',''' ||CHAR(39)|| COALESCE(filter_expression, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(order_by, '')||CHAR(39) ||''',''' || CHAR(39)||COALESCE(windows_spec,'')||CHAR(39) ||''');'  FROM metastore.pipeline_projection where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        curs.execute(sql_projection_node)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--join_node--")
        #sql_join_node="""select 'INSERT INTO pipeline_join (id, filter_expression,join_predicate, join_tables, order_by)  VALUES (' || id || ',''' || COALESCE(filter_expression, '') || ''',''' || COALESCE(join_predicate, '') ||''',''' || COALESCE(join_tables,'') ||''',''' || COALESCE(order_by,'') ||''');'  FROM metastore.pipeline_join where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name='%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        sql_join_node="""select 'INSERT INTO pipeline_join (id, filter_expression,join_predicate, join_tables, order_by)  VALUES (' || id || ',''' ||CHAR(39)|| COALESCE(filter_expression, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(join_predicate, '')||CHAR(39) ||''',''' ||CHAR(39)|| COALESCE(join_tables,'')||CHAR(39) ||''',''' ||CHAR(39)|| COALESCE(order_by,'')||CHAR(39) ||''');'  FROM metastore.pipeline_join where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name='%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        curs.execute(sql_join_node)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--aggregation_node--")
        #sql_aggregation_node="""select 'INSERT INTO pipeline_aggregation (id, filter_expression,group_expression, having_expression, order_by)  VALUES (' || id || ',''' || COALESCE(filter_expression, '') || ''',''' || COALESCE(group_expression, '') ||''',''' || COALESCE(having_expression,'') ||''',''' || COALESCE(order_by,'') ||''');'  FROM metastore.pipeline_aggregation where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        sql_aggregation_node="""select 'INSERT INTO pipeline_aggregation (id, filter_expression,group_expression, having_expression, order_by)  VALUES (' || id || ',''' ||CHAR(39)|| COALESCE(filter_expression, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(group_expression, '')||CHAR(39) ||''',''' ||CHAR(39)|| COALESCE(having_expression,'')||CHAR(39) ||''',''' ||CHAR(39)|| COALESCE(order_by,'')||CHAR(39) ||''');'  FROM metastore.pipeline_aggregation where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        curs.execute(sql_aggregation_node)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--sink_node--")
        sql_sink_node="""select 'INSERT INTO pipeline_sink (id, sink_type)  VALUES (' || id || ',''' ||char(39)|| COALESCE(sink_type, '')||char(39) ||''');'  FROM metastore.pipeline_sink where id in(select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from  metastore.pipeline_container where name='%s') and case_sensitive_name = '%s'))"""%(container_name, pipeline_name)
        curs.execute(sql_sink_node)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])
            
        if (option ==4):
            
            print("\n--DQ_pipeline--")
            sql_DQ_pipeline = """select 'INSERT INTO pipeline_simple_dq (id) VALUES ('||id||');' FROM metastore.pipeline_simple_dq where id in (select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name='%s') and name = '%s'))"""%(container_name,pipeline_name)
            curs.execute(sql_DQ_pipeline)
            res = curs.fetchall()
            for i in range(len(res)):
                list_queries.append(res[i][0])

            print("\n--pipeline_simple_dq_rule--")
            #sql_pipeline_simple_dq_rule = """SELECT 'INSERT INTO pipeline_simple_dq_rule (id, columns ,expression , filter , name,fk_pipeline_simple_dq_id  ) VALUES (' || id || ',''' || COALESCE(columns, '') || ''',''' || COALESCE(expression, '') || ''',''' || COALESCE(filter, '') || ''',''' || COALESCE(name, '') || ''',' || COALESCE(fk_pipeline_simple_dq_id, 0) || ');' FROM metastore.pipeline_simple_dq_rule where fk_pipeline_simple_dq_id in (select id from metastore.pipeline_simple_dq where id in (select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s')and name='%s')))"""%(container_name,pipeline_name)
            sql_pipeline_simple_dq_rule = """SELECT 'INSERT INTO pipeline_simple_dq_rule (id, columns ,expression , filter , name,fk_pipeline_simple_dq_id  ) VALUES (' || id || ',''' ||CHAR(39)||COALESCE(columns, '')||CHAR(39)|| ''',''' ||CHAR(39)||COALESCE(expression, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(filter, '')||CHAR(39) || ''',''' ||CHAR(39)||COALESCE(name, '')||CHAR(39)|| ''',' || COALESCE(fk_pipeline_simple_dq_id, 0) || ');' FROM metastore.pipeline_simple_dq_rule where fk_pipeline_simple_dq_id in (select id from metastore.pipeline_simple_dq where id in (select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s')and name='%s')))"""%(container_name,pipeline_name)

            curs.execute(sql_pipeline_simple_dq_rule)
            res = curs.fetchall()
            for i in range(len(res)):
                list_queries.append(res[i][0])
           
        
        for i in range(len(list_queries)):
            print("\n")
            print(list_queries[i])   
        generateSQLfile(list_queries,'',pipeline_name,'Pipeline')
        mainMenu(con)
        
def PermanentViews_InsertQueries(con):
    
    View_name = (input("\n Enter the Permanent View name: ")) 
    View_name = View_name.upper().strip()
    chk_viewname= chk_view_name(View_name,con)
    if chk_viewname == False:
        print("\n Incorrect view name.")  
        mainMenu(con)
    else:
        curs = con.cursor()
        list_queries = []

        print("\n--schema_store_view--")
        sql_Permanent_View = """select 'INSERT INTO schema_store_view (id, description ,generator, materialized_table,name,query,fk_organisation_id   )  VALUES (' || id || ',''' || CHAR(39) || COALESCE(description, '') || CHAR(39) || ''',''' || CHAR(39) || COALESCE(generator, '') || char(39) || ''',''' || CHAR(39) || COALESCE(materialized_table, '') || CHAR(39) || ''',''' || CHAR(39) || COALESCE(name, '') || CHAR(39) || ''',''' || CHAR(39) || COALESCE(query, '') || CHAR(39) || ''',' || COALESCE(fk_organisation_id, 0) || ');' FROM metastore.schema_store_view where name = '%s'"""%(View_name)
        curs.execute(sql_Permanent_View)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--schema_store_view_schema--")
        sql_schema_store_view_schema = """select  'INSERT INTO schema_store_view_schema  (id, column_name  ,data_type , fk_schema_store_view_id) VALUES (' || id || ',''' || CHAR(39) || COALESCE(column_name, '') || CHAR(39) || ''',''' || CHAR(39) || COALESCE(data_type, '') || char(39) || ''',' || COALESCE(fk_schema_store_view_id, 0) || ');' FROM metastore.schema_store_view_schema where fk_schema_store_view_id in (select id from metastore.schema_store_view where name = '%s' )"""%(View_name)
        curs.execute(sql_schema_store_view_schema)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        for i in range(len(list_queries)):
                print("\n")
                print(list_queries[i])   

        generateSQLfile(list_queries,'',View_name,'Permanent View')
        mainMenu(con)
    
# def DQContainer_InsertQueries(con):
    
#     container_name = (input("\n Enter the container name: "))
#     container_name = container_name.lower().strip()
#     #pipeline_name = (input("Enter the pipeline name: "))
#     #pipeline_name = pipeline_name.lower().strip()
#     curs = con.cursor()
#     list_queries = []


#     print("\n--DQ_pipeline--")
#     sql_DQ_pipeline = """select 'INSERT INTO pipeline_simple_dq (id) VALUES ('||id||');' FROM metastore.pipeline_simple_dq where id in (select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name='%s')))"""%(container_name)

#     curs.execute(sql_DQ_pipeline)
#     res = curs.fetchall()
#     for i in range(len(res)):
#         list_queries.append(res[i][0])

#     print("\n--pipeline_simple_dq_rule--")
#     #sql_pipeline_simple_dq_rule = """SELECT 'INSERT INTO pipeline_simple_dq_rule (id, columns ,expression , filter , name,fk_pipeline_simple_dq_id  ) VALUES (' || id || ',''' || COALESCE(columns, '') || ''',''' || COALESCE(expression, '') || ''',''' || COALESCE(filter, '') || ''',''' || COALESCE(name, '') || ''',' || COALESCE(fk_pipeline_simple_dq_id, 0) || ');' FROM metastore.pipeline_simple_dq_rule where fk_pipeline_simple_dq_id in (select id from metastore.pipeline_simple_dq where id in (select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s'))))"""%(container_name)
#     sql_pipeline_simple_dq_rule = """SELECT 'INSERT INTO pipeline_simple_dq_rule (id, columns ,expression , filter , name,fk_pipeline_simple_dq_id  ) VALUES (' || id || ',''' ||CHAR(39)||COALESCE(columns, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(expression, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(filter, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(name, '')||CHAR(39) || ''',' || COALESCE(fk_pipeline_simple_dq_id, 0) || ');' FROM metastore.pipeline_simple_dq_rule where fk_pipeline_simple_dq_id in (select id from metastore.pipeline_simple_dq where id in (select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s'))))"""%(container_name)
#     curs.execute(sql_pipeline_simple_dq_rule)
#     res = curs.fetchall()
#     for i in range(len(res)):
#         list_queries.append(res[i][0])

#     for i in range(len(list_queries)):
#         print("\n")
#         print(list_queries[i])
#     generateSQLfile(list_queries)
#     mainMenu(con)
        
# def DQIndividualPipeline_InsertQueries(con):
#     container_name = (input("Enter the container name: "))
#     container_name = container_name.lower().strip()
#     pipeline_name = (input("Enter the pipeline name: "))
#     pipeline_name = pipeline_name.lower().strip()
#     curs = con.cursor()
#     list_queries = []


#     print("\n--DQ_pipeline--")
#     sql_DQ_pipeline = """select 'INSERT INTO pipeline_simple_dq (id) VALUES ('||id||');' FROM metastore.pipeline_simple_dq where id in (select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name='%s') and name = '%s'))"""%(container_name,pipeline_name)

#     curs.execute(sql_DQ_pipeline)
#     res = curs.fetchall()
#     for i in range(len(res)):
#         list_queries.append(res[i][0])

#     print("\n--pipeline_simple_dq_rule--")
#     #sql_pipeline_simple_dq_rule = """SELECT 'INSERT INTO pipeline_simple_dq_rule (id, columns ,expression , filter , name,fk_pipeline_simple_dq_id  ) VALUES (' || id || ',''' || COALESCE(columns, '') || ''',''' || COALESCE(expression, '') || ''',''' || COALESCE(filter, '') || ''',''' || COALESCE(name, '') || ''',' || COALESCE(fk_pipeline_simple_dq_id, 0) || ');' FROM metastore.pipeline_simple_dq_rule where fk_pipeline_simple_dq_id in (select id from metastore.pipeline_simple_dq where id in (select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s')and name='%s')))"""%(container_name,pipeline_name)
#     sql_pipeline_simple_dq_rule = """SELECT 'INSERT INTO pipeline_simple_dq_rule (id, columns ,expression , filter , name,fk_pipeline_simple_dq_id  ) VALUES (' || id || ',''' ||CHAR(39)||COALESCE(columns, '')||CHAR(39)|| ''',''' ||CHAR(39)||COALESCE(expression, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(filter, '')||CHAR(39) || ''',''' ||CHAR(39)||COALESCE(name, '')||CHAR(39)|| ''',' || COALESCE(fk_pipeline_simple_dq_id, 0) || ');' FROM metastore.pipeline_simple_dq_rule where fk_pipeline_simple_dq_id in (select id from metastore.pipeline_simple_dq where id in (select id from metastore.pipeline_node where fk_pipeline_relation_id in (select id from metastore.pipeline_relation where fk_pipeline_container_id in (select id from metastore.pipeline_container where name = '%s')and name='%s')))"""%(container_name,pipeline_name)

#     curs.execute(sql_pipeline_simple_dq_rule)
#     res = curs.fetchall()
#     for i in range(len(res)):
#         list_queries.append(res[i][0])

#     for i in range(len(list_queries)):
#         print("\n")
#         print(list_queries[i])
#     generateSQLfile(list_queries)
#     mainMenu(con)
    
def DataMart_InsertQueries(con):
    datamart_name = (input("Enter the data mart name: "))
    datamart_name = datamart_name.upper().strip()
    #chk_datamart_name(datamart_name,con)
    chk_dm_name= chk_datamart_name(datamart_name,con)
    if chk_dm_name == False:
        print("\n Incorrect data mart name.")  
        mainMenu(con)
    else:
        curs = con.cursor()
        list_queries = []

        print("\n--data_mart--")
        #sql_data_mart = """SELECT 'INSERT INTO data_mart (id, case_sensitive_name ,default_view , name , fk_organisation_id )  VALUES (' || id || ',''' || COALESCE(case_sensitive_name, '') || ''',''' || COALESCE(default_view, '') ||''',''' || COALESCE(name,'') ||''','|| COALESCE(fk_organisation_id,'') ||');' FROM metastore.data_mart where name= '%s';"""%(datamart_name)
        sql_data_mart = """SELECT 'INSERT INTO data_mart (id, case_sensitive_name ,default_view , name , fk_organisation_id ) VALUES (' || id || ',''' || CHAR(39)|| COALESCE(case_sensitive_name, '') ||CHAR(39)|| ''',''' ||CHAR(39)|| COALESCE(default_view, '')||CHAR(39)||''',''' ||CHAR(39)|| COALESCE(name,'')||CHAR(39) ||''','|| COALESCE(fk_organisation_id,'') ||');' FROM metastore.data_mart where name= '%s';"""%(datamart_name)
        curs.execute(sql_data_mart)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--data_mart_table--")
        #sql_data_mart_table = """SELECT 'INSERT INTO data_mart_table (id, case_sensitive_name , name , source_table  ,fk_data_mart_id  )  VALUES (' || id || ',''' || COALESCE(case_sensitive_name, '') || ''',''' || COALESCE(name, '') ||''',''' || COALESCE(source_table ,'') ||''','|| COALESCE(fk_data_mart_id,0) ||');'  FROM metastore.data_mart_table where fk_data_mart_id in (select id from  metastore.data_mart where name = '%s');"""%(datamart_name)
        sql_data_mart_table = """SELECT 'INSERT INTO data_mart_table (id, case_sensitive_name , name , source_table  ,fk_data_mart_id  )  VALUES (' || id || ',''' ||CHAR(39)|| COALESCE(case_sensitive_name, '')||CHAR(39)|| ''',''' ||CHAR(39)|| COALESCE(name, '')||CHAR(39)||''',''' ||CHAR(39)||COALESCE(source_table ,'')||CHAR(39) ||''','|| COALESCE(fk_data_mart_id,0) ||');'  FROM metastore.data_mart_table where fk_data_mart_id in (select id from  metastore.data_mart where name = '%s');"""%(datamart_name)
        curs.execute(sql_data_mart_table)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--data_mart_table_schema--")
        #sql_data_mart_table_schema = """select  'INSERT INTO data_mart_table_schema (id, real_column_name , virtual_name, fk_data_mart_table_id)  VALUES (' || id || ',''' || COALESCE(real_column_name, '') || ''',''' || COALESCE(virtual_name, '') ||''','|| COALESCE(fk_data_mart_table_id,0) ||');' from metastore.data_mart_table_schema where fk_data_mart_table_id in (select id from metastore.data_mart_table where fk_data_mart_id in (select id from  metastore.data_mart where name = '%s'));"""%(datamart_name)
        sql_data_mart_table_schema = """select  'INSERT INTO data_mart_table_schema (id, real_column_name , virtual_name, fk_data_mart_table_id)  VALUES (' || id || ',''' ||CHAR(39)||COALESCE(real_column_name, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(virtual_name, '')||CHAR(39) ||''','|| COALESCE(fk_data_mart_table_id,0) ||');' from metastore.data_mart_table_schema where fk_data_mart_table_id in (select id from metastore.data_mart_table where fk_data_mart_id in (select id from  metastore.data_mart where name = '%s'));"""%(datamart_name)
        curs.execute(sql_data_mart_table_schema)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        for i in range(len(list_queries)):
            print("\n")
            print(list_queries[i])
        generateSQLfile(list_queries,datamart_name,'','DataMart')
        mainMenu(con)
        
def DataMart_IndTable_InsertQueries(con):
    datamart_name = (input("Enter the data mart name: "))
    datamart_name = datamart_name.upper().strip()
    table_name = (input("Enter the Table/View name: "))
    table_name = table_name.upper().strip()
    chk_dm_name= chk_datamart_table_name(datamart_name,table_name,con)
    if chk_dm_name == False:
        print("\n Incorrect data mart name.")  
        mainMenu(con)
    else:
        curs = con.cursor()
        list_queries = []

        print("\n--data_mart_table--")
        #sql_data_mart = """SELECT 'INSERT INTO data_mart (id, case_sensitive_name ,default_view , name , fk_organisation_id )  VALUES (' || id || ',''' || COALESCE(case_sensitive_name, '') || ''',''' || COALESCE(default_view, '') ||''',''' || COALESCE(name,'') ||''','|| COALESCE(fk_organisation_id,'') ||');' FROM metastore.data_mart where name= '%s';"""%(datamart_name)
        sql_data_mart_table = """SELECT 'INSERT INTO data_mart_table (id, case_sensitive_name , name , source_table  ,fk_data_mart_id  )  VALUES (' || id || ',''' || CHAR(39) || COALESCE(case_sensitive_name, '') || CHAR(39) || ''',''' || CHAR(39) || COALESCE(name, '') || CHAR(39) || ''',''' || CHAR(39) || COALESCE(source_table, '') || CHAR(39) || ''',' || COALESCE(fk_data_mart_id, 0) || ');' FROM metastore.data_mart_table where fk_data_mart_id in ( select id from metastore.data_mart where name = '%s') and name='%s';"""%(datamart_name,table_name)
        curs.execute(sql_data_mart_table)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        print("\n--data_mart_table_schema--")
        #sql_data_mart_table_schema = """select  'INSERT INTO data_mart_table_schema (id, real_column_name , virtual_name, fk_data_mart_table_id)  VALUES (' || id || ',''' || COALESCE(real_column_name, '') || ''',''' || COALESCE(virtual_name, '') ||''','|| COALESCE(fk_data_mart_table_id,0) ||');' from metastore.data_mart_table_schema where fk_data_mart_table_id in (select id from metastore.data_mart_table where fk_data_mart_id in (select id from  metastore.data_mart where name = '%s'));"""%(datamart_name)
        sql_data_mart_table_schema = """select  'INSERT INTO data_mart_table_schema (id, real_column_name , virtual_name, fk_data_mart_table_id)  VALUES (' || id || ',''' ||CHAR(39)|| COALESCE(real_column_name, '')||CHAR(39) || ''',''' ||CHAR(39)|| COALESCE(virtual_name, '')||CHAR(39) ||''','|| COALESCE(fk_data_mart_table_id,0) ||');' from metastore.data_mart_table_schema where fk_data_mart_table_id in (select id from metastore.data_mart_table where fk_data_mart_id in (select id from  metastore.data_mart where name = '%s')and name='%s');"""%(datamart_name,table_name)
        curs.execute(sql_data_mart_table_schema)
        res = curs.fetchall()
        for i in range(len(res)):
            list_queries.append(res[i][0])

        for i in range(len(list_queries)):
            print("\n")
            print(list_queries[i])
        generateSQLfile(list_queries,datamart_name,'','DataMart')
        mainMenu(con)
    
def generateSQLfile(list_queries,filename1,filename2,filename3):
    #df = pd.DataFrame (list_queries, columns = [''])
    #print("process start")
    #print(df)
    #df.to_csv('output.sql', sep='\t', index=False,header= False) 
    #df.to_csv('output.txt', sep='\t', index=False,header= False,quoting=None) 
    filename1=filename1.lower()
    filename2=filename2.lower()
    output_filename =filename1+filename2+ '-'+ filename3 +'.sql'
    df = open(output_filename, 'w')
    for i in range(len(list_queries)):
        df.write(list_queries[i])
        df.write('\n')
    df.close()
    
   
    #df.to_csv('output.sql', sep=' ', index=False,header= False)       
    
    
con = serverDetails()
mainMenu(con)

#host = "zetaris-test.mcri.edu.au"
#port = "10000"
#uid = "net.support@mcri.edu.au"
#pwd = "57vrvXV0b5@"
#migration_container_test
#visit_detail_migration_test
#visit_occurence_test
#visit_detail_migration_test 

*****Enter the server details*****

Enter the host name: zetaris-test.mcri.edu.au
Enter the port name: 10000
Enter the username name: net.support@mcri.edu.au
Enter the password: ········

 Connection successfull

 Select the following option to migrate: 

1. Data Pipeline container
2. Individual Data Pipeline
3. Data Quality container
4. Individual Data Quality Pipeline
5. Data Mart
6. Individual Table/View in the existing Datamart
7. Permanent Views
8. Exit

Enter your selection: 1

Enter the container name: migration_container_test_1

--pipeline_container--

--pipeline_node_relation--

--pipeline_node--

--pipeline_node_schema--

--pipeline_node_property--

--Datasource_Node--

--sql_table--

--projection_node--

--join_node--

--aggregation_node--

--sink_node--


INSERT INTO pipeline_container  (id, case_sensitive_name,description, name, fk_organisation_id)  VALUES (3987921,'MIGRATION_CONTAINER_TEST_1','','migration_container_test_1',23);


INSERT INTO pipeline_relation (id, case_s